# Module 2. Personalize 솔류션 생성 및 평가 지표 확인하기

이 노트북은 Module1에서 생성한 데이타셋 그룹, 데이타셋을 바탕으로 아래와 같은 작업을 합니다.

* 레서피(알고리즘) 선택 및 솔류션 생성
* 솔류션 버전 생성
* 솔류션 평가 지표 얻기



## 라이브러리 임포트

파이썬에는 광범위한 라이브러리 모음이 포함되어 있으며, 본 핸즈온을 위해서 핵심 데이터 과학 도구인 boto3 (AWS SDK) 및 Pandas/Numpy와 같은 라이브러리를 가져와야 합니다.

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time

다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [3]:
%store -r

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

In [4]:
suffix = str(np.random.uniform())[4:9]

## 솔루션 및 버전 생성

Amazon Personalize에서 훈련된 모델을 솔루션이라고 하며, 각 솔루션에는 모델이 훈련되었을 때 주어진 데이터량과 관련된 많은 특정 버전들이 있을 수 있습니다.

우선, Amazon Personalize에서 지원되는 모든 레시피(레시피는 아직 데이터에 대해 훈련되지 않은 알고리즘입니다.)들을 나열합니다. 리스트업된 레시피들 중 하나를 선택하고 이를 사용하여 모델을 빌드해 보세요.

이 LAB에서 아래와 같은 솔루션을 생성하여 성능을 비교하여 봅니다.

    1) HRNN
    2) HRNN Coldstart
    3) SIMS
    4) Personalized Ranking

이 프로세스의 완료는 실제로 40분 이상 소요됩니다. 작업이 완료될 때까지(즉, 활성화 상태가 될 때까지) while 루프를 수행하는 방법도 있지만, 이렇게 하면 다른 셀의 실행을 차단하게 됩니다. 따라서, 많은 모델을 만들어 신속하게 배포하려면 while 루프를 사용하는 대신, 필요한 솔루션 버전들을 생성 후, SageMaker 및 Cloudwatch에서 업데이트를 확인하세요.

###  레시피 리스트 확인

In [5]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

arn:aws:personalize:::recipe/aws-hrnn
arn:aws:personalize:::recipe/aws-hrnn-coldstart
arn:aws:personalize:::recipe/aws-hrnn-metadata
arn:aws:personalize:::recipe/aws-personalized-ranking
arn:aws:personalize:::recipe/aws-popularity-count
arn:aws:personalize:::recipe/aws-sims
arn:aws:personalize:::recipe/aws-user-personalization


### HRNN 솔루션 생성

In [6]:
 # Recipe 선택 
hrnn_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

In [7]:
# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Demo-hrnn-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = hrnn_recipe_arn,
)

hrnn_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-hrnn-59052",
  "ResponseMetadata": {
    "RequestId": "db8dd123-3870-443a-b65f-6524521fffa1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 08 Jun 2020 03:17:02 GMT",
      "x-amzn-requestid": "db8dd123-3870-443a-b65f-6524521fffa1",
      "content-length": "90",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [8]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_solution_arn
)

hrnn_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-hrnn-59052/d4eb6b82",
  "ResponseMetadata": {
    "RequestId": "1a21ef1c-757c-47d2-9ebb-b089ef40938e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 08 Jun 2020 03:17:03 GMT",
      "x-amzn-requestid": "1a21ef1c-757c-47d2-9ebb-b089ef40938e",
      "content-length": "106",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### HRNN ColdStart 솔루션 생성

In [9]:
 # Recipe 선택 
hrnn_coldstart_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn-coldstart"

In [10]:
# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Demo-hrnn-coldstart-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = hrnn_coldstart_recipe_arn,
    solutionConfig = {
        "featureTransformationParameters" : {
            'cold_start_max_duration' : '5',
            'cold_start_relative_from' : 'latestItem',
            'cold_start_max_interactions':'15'
        }
    }
)

hrnn_coldstart_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-hrnn-coldstart-59052",
  "ResponseMetadata": {
    "RequestId": "fa3530e5-c8db-49ad-847d-6e12de78f6b5",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 08 Jun 2020 03:17:05 GMT",
      "x-amzn-requestid": "fa3530e5-c8db-49ad-847d-6e12de78f6b5",
      "content-length": "100",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [11]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_coldstart_solution_arn
)

hrnn_coldstart_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-hrnn-coldstart-59052/ae7ce932",
  "ResponseMetadata": {
    "RequestId": "af0c5c5b-70fc-40f6-bcd9-f89da19e6340",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 08 Jun 2020 03:17:06 GMT",
      "x-amzn-requestid": "af0c5c5b-70fc-40f6-bcd9-f89da19e6340",
      "content-length": "116",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [12]:
'''
# Recipe 선택 
hrnn_coldstart_t3_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn-coldstart"
# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Demo-hrnn-coldstart-t3-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = hrnn_coldstart_t3_recipe_arn,
    performHPO = True,
    solutionConfig = {
        "featureTransformationParameters" : {
            'cold_start_max_duration' : '5',
            'cold_start_relative_from' : 'latestItem',
            'cold_start_max_interactions':'15'
            
        }
    }
)

hrnn_coldstart_t3_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_coldstart_t3_solution_arn
)

hrnn_coldstart_t3_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))
'''

'\n# Recipe 선택 \nhrnn_coldstart_t3_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn-coldstart"\n# Solution 생성 \ncreate_solution_response = personalize.create_solution(\n    name = "Demo-hrnn-coldstart-t3-" + suffix,\n    datasetGroupArn = dataset_group_arn,\n    recipeArn = hrnn_coldstart_t3_recipe_arn,\n    performHPO = True,\n    solutionConfig = {\n        "featureTransformationParameters" : {\n            \'cold_start_max_duration\' : \'5\',\n            \'cold_start_relative_from\' : \'latestItem\',\n            \'cold_start_max_interactions\':\'15\'\n            \n        }\n    }\n)\n\nhrnn_coldstart_t3_solution_arn = create_solution_response[\'solutionArn\']\nprint(json.dumps(create_solution_response, indent=2))\n\n# 솔루션 버전 생성\ncreate_solution_version_response = personalize.create_solution_version(\n    solutionArn = hrnn_coldstart_t3_solution_arn\n)\n\nhrnn_coldstart_t3_solution_version_arn = create_solution_version_response[\'solutionVersionArn\']\nprint(json.dumps(create_

### SIMS 솔루션 생성 

In [13]:
 # Recipe 선택 
sims_recipe_arn = "arn:aws:personalize:::recipe/aws-sims"

In [14]:
# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Demo-sims-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = sims_recipe_arn
)

sims_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-sims-59052",
  "ResponseMetadata": {
    "RequestId": "e131d1ed-55c8-4ad0-a7f9-fe83d0852b29",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 08 Jun 2020 03:17:08 GMT",
      "x-amzn-requestid": "e131d1ed-55c8-4ad0-a7f9-fe83d0852b29",
      "content-length": "90",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [15]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)
sims_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-sims-59052/1a631002",
  "ResponseMetadata": {
    "RequestId": "5a229a36-9e58-4440-a4ca-49947550ac1d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 08 Jun 2020 03:17:09 GMT",
      "x-amzn-requestid": "5a229a36-9e58-4440-a4ca-49947550ac1d",
      "content-length": "106",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Personalize Ranking 솔루션 생성 

In [16]:
 # Recipe 선택 
ranking_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

In [17]:
# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Demo-ranking-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = ranking_recipe_arn
)

ranking_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-ranking-59052",
  "ResponseMetadata": {
    "RequestId": "2f007387-022f-438a-98be-aad93204200a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 08 Jun 2020 03:17:11 GMT",
      "x-amzn-requestid": "2f007387-022f-438a-98be-aad93204200a",
      "content-length": "93",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [18]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = ranking_solution_arn
)
ranking_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-ranking-59052/e8db7ab9",
  "ResponseMetadata": {
    "RequestId": "42eabb10-9b52-4ae4-8f64-21504f370d4c",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 08 Jun 2020 03:17:12 GMT",
      "x-amzn-requestid": "42eabb10-9b52-4ae4-8f64-21504f370d4c",
      "content-length": "109",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 모델 생성 확인 하기 
아래 코드 실행하여 모델 생성 여부를 확인합니다. 

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    #hrnn status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = hrnn_solution_version_arn
    )  
    status_hrnn = describe_solution_version_response["solutionVersion"]["status"]
    print("HRNN SolutionVersion: {}".format(status_hrnn))
    
    #hrnn_coldstart status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = hrnn_coldstart_solution_version_arn
    )
    status_hrnn_cs = describe_solution_version_response["solutionVersion"]["status"]    
    print("HRNN ColdStart SolutionVersion: {}".format(status_hrnn_cs))
    
    #sims status
    describe_solution_version_response = personalize.describe_solution_version(
    solutionVersionArn = sims_solution_version_arn
    )
    status_sims = describe_solution_version_response["solutionVersion"]["status"]
    print("Sims SolutionVersion: {}".format(status_sims))
    
    #ranking status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = ranking_solution_version_arn
    )
    status_ranking= describe_solution_version_response["solutionVersion"]["status"]    
    print("Ranking SolutionVersion: {}".format(status_ranking))    
    
    if (status_hrnn == "ACTIVE" or status_hrnn == "CREATE FAILED") &\
       (status_hrnn_cs == "ACTIVE" or status_hrnn_cs == "CREATE FAILED") &\
       (status_sims == "ACTIVE" or status_sims == "CREATE FAILED") &\
       (status_ranking == "ACTIVE" or status_ranking == "CREATE FAILED"):
        break
    print("-------------------------------------->")
    time.sleep(60)

print("All solution creation completed")

HRNN SolutionVersion: CREATE PENDING
HRNN ColdStart SolutionVersion: CREATE PENDING
Sims SolutionVersion: CREATE PENDING
Ranking SolutionVersion: CREATE PENDING
-------------------------------------->
HRNN SolutionVersion: CREATE IN_PROGRESS
HRNN ColdStart SolutionVersion: CREATE IN_PROGRESS
Sims SolutionVersion: CREATE IN_PROGRESS
Ranking SolutionVersion: CREATE IN_PROGRESS
-------------------------------------->
HRNN SolutionVersion: CREATE IN_PROGRESS
HRNN ColdStart SolutionVersion: CREATE IN_PROGRESS
Sims SolutionVersion: CREATE IN_PROGRESS
Ranking SolutionVersion: CREATE IN_PROGRESS
-------------------------------------->
HRNN SolutionVersion: CREATE IN_PROGRESS
HRNN ColdStart SolutionVersion: CREATE IN_PROGRESS
Sims SolutionVersion: CREATE IN_PROGRESS
Ranking SolutionVersion: CREATE IN_PROGRESS
-------------------------------------->
HRNN SolutionVersion: CREATE IN_PROGRESS
HRNN ColdStart SolutionVersion: CREATE IN_PROGRESS
Sims SolutionVersion: CREATE IN_PROGRESS
Ranking Solutio

## 솔루션 평가 지표 얻기

이번 파트에서는 Amazon Personalize에서 기본으로 제공하는 솔루션에 대한 평가 지표를 확인해 봅니다. 
Amazon Personalize에서는 평가 지표를 생성하기 위해 약 랜덤으로 10% 사용자의 interaction data를 테스트 용으로 활용합니다. 

아래 이미지는 Amazon Personalize가 데이터를 분리하는 방법을 보여줍니다. 사용자가 10 명이고 각각 10 개의 상호 작용이있는 경우 (여기에서 원은 Interaction data를 나타냄) 타임 스탬프를 기준으로 가장 오래된 것부터 최신 것까지 나열된 것입니다. Amazon Personalize는 사용자의 90 % (파란색 원)의 모든 Interaction 데이터를 사용하여 솔루션 버전을 훈련시키고 나머지 10 %는 평가를 위해 사용합니다. 나머지 10 %의 각 사용자에 대해 Interaction data (녹색 원)의 90 %가 훈련 된 모델의 입력값으로 사용됩니다. 데이터의 나머지 10 % (주황색 원)는 모델에서 생성 된 추천 결과물과 비교되고 평가 지표를 계산하는 데 사용됩니다.



![personalize metrics](static/imgs/personalize_metrics.png)

[솔류션 평가 지표 정의](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html)
는 개발자 문서의 링크 참조 바랍니다. 또한 이 링크 [솔류션 평가 정의 예제](http://francescopochetti.com/recommend-expedia-hotels-with-amazon-personalize-the-magic-of-hierarchical-rnns/) 의 페이지 맨 아래 쪽을 보시면 조금 더 직관적인 그림을 보실 수 있습니다.
 <br>
또한 reciprocal_rank_at_5, normalized_discounted_cumulative_gain_at_5,precision_at_5 의 예제는 아래와 같습니다. 
* Exmaple
    * 5 개의 추천리스트를 제공했고, 이 중에 2번째와 5번째가 실제 데이타와 일치 했다고 하면, 쉽게 이렇게 [0,1,0,0,1] 표시 할 수 있습니다.
        * reciprocal_rank
            * 1/2 (0.5) # 가장 빠른 순서의 하나만을 선택 합니다
        * normalized_discounted_cumulative_gain_at_5
            * (1/log(1+2) + 1/log(1+5)) / (1/log(1+1) + 1/log(1+2)) = 0.6241
        * precision_at_5
            * 2/5 (0.4)



#### 조금더 상세하고 Custum 평가 지표를 얻기 위해서 이전에 분리해둔 테스트 데이터를 가지고 캠페인 생성 후 별도 테스트를 진행하도록 합니다.



In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = hrnn_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = hrnn_coldstart_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = sims_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = ranking_solution_version_arn 
)

print(json.dumps(get_solution_metrics_response, indent=2))

## 리뷰

이번 실습에서는 Amazon Personalize에서 제공하는 다양한 솔루션을 생성하고 Amazon Personalize에서 제공하는 솔루션 버전을 생성해 보았습니다.다음 LAB에서는 (`3.Deploying_Campaigns.ipynb`) 방금 생성한 솔루션을 서비스에 배포 하기 위해 Campaing을 배포해 보도록 합니다. 


In [ ]:
%store hrnn_solution_version_arn
%store hrnn_solution_arn
%store hrnn_coldstart_solution_version_arn
%store hrnn_coldstart_solution_arn
%store sims_solution_version_arn
%store sims_solution_arn
%store ranking_solution_version_arn
%store ranking_solution_arn
